In [1]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding="latin-1")
df = df[["v1", "v2"]]
df.columns = ["label", "message"]

X = df["message"]
y = df["label"]


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
from features import get_vectorizer

vectorizer = get_vectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)


In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_vec, y_train)

y_pred = model_tfidf.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9713004484304932


In [7]:
type(vectorizer)
model_tfidf.n_features_in_


1000

In [9]:

model_tfidf.predict(
    vectorizer.transform(["Congratulations! You have won a free prize"])
)


array(['spam'], dtype='<U4')

In [11]:
import pickle

pickle.dump(model_tfidf, open("spam_model.pkl", "wb"))
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))


In [12]:
model_tfidf.n_features_in_


1000